In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15,15
import seaborn as sns

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier

from sklearn.model_selection import train_test_split, GridSearchCV , cross_val_score,GroupShuffleSplit
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier , ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score,accuracy_score ,confusion_matrix,mean_absolute_error
from sklearn.preprocessing import PolynomialFeatures
import warnings
warnings.filterwarnings("ignore")

from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import ClusterCentroids , NearMiss

from tqdm.notebook import tqdm ,tnrange

/kaggle/input/rapids/rapids.0.15.0
/kaggle/input/rapids/rapids.0.14.0
/kaggle/input/healthcareanalyticsii/train_data_dict.csv
/kaggle/input/healthcareanalyticsii/sample_submission.csv
/kaggle/input/healthcareanalyticsii/test.csv
/kaggle/input/healthcareanalyticsii/train.csv


In [2]:
train_data = pd.read_csv('../input/healthcareanalyticsii/train.csv')
test_data = pd.read_csv('../input/healthcareanalyticsii/test.csv')

In [3]:
print(train_data.shape)
train_data.head()

(318438, 18)


,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay
0,1,8,c,3,Z,3,radiotherapy,R,F,2.0,31397,7.0,Emergency,Extreme,2,51-60,4911.0,0-10
1,2,2,c,5,Z,2,radiotherapy,S,F,2.0,31397,7.0,Trauma,Extreme,2,51-60,5954.0,41-50
2,3,10,e,1,X,2,anesthesia,S,E,2.0,31397,7.0,Trauma,Extreme,2,51-60,4745.0,31-40
3,4,26,b,2,Y,2,radiotherapy,R,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,7272.0,41-50
4,5,26,b,2,Y,2,radiotherapy,S,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,5558.0,41-50


In [4]:
print(test_data.shape)
test_data.head()

(137057, 17)


,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit
0,318439,21,c,3,Z,3,gynecology,S,A,2.0,17006,2.0,Emergency,Moderate,2,71-80,3095.0
1,318440,29,a,4,X,2,gynecology,S,F,2.0,17006,2.0,Trauma,Moderate,4,71-80,4018.0
2,318441,26,b,2,Y,3,gynecology,Q,D,4.0,17006,2.0,Emergency,Moderate,3,71-80,4492.0
3,318442,6,a,6,X,3,gynecology,Q,F,2.0,17006,2.0,Trauma,Moderate,3,71-80,4173.0
4,318443,28,b,11,X,2,gynecology,R,F,2.0,17006,2.0,Trauma,Moderate,4,71-80,4161.0


In [5]:
train_data['City_Code_Patient'] = train_data['City_Code_Patient'].fillna(-1)
train_data['Bed Grade'] = train_data['Bed Grade'].fillna(-1)

In [6]:
test_data['City_Code_Patient'] = test_data['City_Code_Patient'].fillna(-1)
test_data['Bed Grade'] = test_data['Bed Grade'].fillna(-1)

In [7]:
cat_cols = ['Hospital_code','Hospital_type_code','City_Code_Hospital','Hospital_region_code'
            ,'Department','Ward_Type','Ward_Facility_Code','Bed Grade','City_Code_Patient',
           # 'Type of Admission','Severity of Illness',
            'Age']

In [8]:
label = 'Stay'

In [9]:
train_data['patientid'].unique().shape

(92017,)

In [10]:
def encode_cat_cols(train, test, cat_cols): #target

    train_df = train_data.copy()
    
    test_df = test_data.copy()
    
    # Making a dictionary to store all the labelencoders for categroical columns to transform them later.
    
    le_dict = {}

    for col in cat_cols:
        
        le = LabelEncoder()
        
        le.fit(train_df[col].unique().tolist() + test_df[col].unique().tolist())
        
        train_df[col] = le.transform(train_df[[col]])
        
        test_df[col] = le.transform(test_df[[col]])

        le_dict[col] = le

    le = LabelEncoder()
    
    train_df[label] = le.fit_transform(train_df[[label]])
    
    le_dict[label] = le
    
    train_df['Type of Admission'] = train_df['Type of Admission'].map({'Urgent':0,'Emergency':1,'Trauma':2})
    
    train_df['Severity of Illness'] = train_df['Severity of Illness'].map({'Minor':0,'Moderate':1,'Extreme':2})
    
    test_df['Type of Admission'] = test_df['Type of Admission'].map({'Urgent':0,'Emergency':1,'Trauma':2})
    
    test_df['Severity of Illness'] = test_df['Severity of Illness'].map({'Minor':0,'Moderate':1,'Extreme':2})
    
    return train_df, test_df, le_dict

In [11]:
train_df, test_df, le_dict = encode_cat_cols(train_data,test_data,cat_cols)

In [12]:
train_df['train'] =1

test_df['train'] = 0

combined_data = pd.concat([train_df,test_df],axis =0)

combined_data = combined_data.reset_index(drop = True)

combined_data["Patient_Count"] = combined_data.groupby('Hospital_code')['patientid'].transform('nunique') #improvement

combined_data["Avg_Extra_Rooms"] = combined_data.groupby('Hospital_code')['Available Extra Rooms in Hospital'].transform('mean')#improvement

combined_data["Avg_Bed_Grade"] = combined_data.groupby('Hospital_code')['Bed Grade'].transform('mean') #improvement

combined_data["No_of_Visits"] = combined_data.groupby('patientid')['patientid'].transform('count') #improvement

combined_data["Type_Admn_Per_Patient"] = combined_data.groupby('patientid')['Type of Admission'].transform('nunique')#improvement

combined_data["No_Hospitals_Visited_Per_Patient"] = combined_data.groupby('patientid')['Hospital_code'].transform('nunique') ### good improvement

combined_data["No_City_Visited_Per_Patient"] = combined_data.groupby('patientid')['City_Code_Hospital'].transform('nunique')## good improvement

combined_data["No_Rooms_Available_Per_Patient"] = combined_data.groupby('patientid')['Available Extra Rooms in Hospital'].transform('nunique')##improvement

combined_data["Sdev_Visitors_Per_Patient"] = combined_data.groupby('patientid')['Visitors with Patient'].transform('std').fillna(-1)

combined_data["Unq_Hospitals_Per_Pateint_Ill_Admn"] = combined_data.groupby(['patientid','Severity of Illness','Type of Admission'])['Hospital_code'].transform('nunique')

combined_data["Unq_City_Per_Pateint_Ill_Admn"] = combined_data.groupby(['patientid','Severity of Illness','Type of Admission'])['City_Code_Hospital'].transform('nunique')

combined_data["Unq_Ward_Per_Pateint_Ill_Admn"] = combined_data.groupby(['patientid','Severity of Illness','Type of Admission'])['Ward_Type'].transform('nunique')

combined_data["Sdev_Rooms_Per_Hospital_Ill_Admn"] = combined_data.groupby(['Hospital_code','Severity of Illness','Type of Admission'])['Available Extra Rooms in Hospital'].transform('std').fillna(-1)

combined_data["Sdev_Bed_Per_Hospital_Ill_Admn"] = combined_data.groupby(['Hospital_code','Severity of Illness','Type of Admission'])['Bed Grade'].transform('std').fillna(-1)

combined_data["Sdev_Deposit_Per_Hospital_Ill_Admn"] = combined_data.groupby(['Hospital_code','Severity of Illness','Type of Admission'])['Admission_Deposit'].transform('std').fillna(-1)

combined_data["Avg_Rooms_Per_Hospital_Ill_Admn"] = combined_data.groupby(['Hospital_code','Severity of Illness','Type of Admission'])['Available Extra Rooms in Hospital'].transform('mean')

htc_vc = combined_data['Hospital_type_code'].value_counts(normalize=True).to_dict()

#combined_data['HTC_Count'] = combined_data['Hospital_type_code'].map(htc_vc)

ccp_vc = combined_data['City_Code_Patient'].value_counts(normalize=True).to_dict()

combined_data['CCP_Count'] = combined_data['City_Code_Patient'].map(ccp_vc)

In [13]:
combined_data['city_change'] = [0] *len(combined_data) 

combined_data['dep_change'] = [0] *len(combined_data) 

combined_data['hos_type_change'] = [0] *len(combined_data) 

combined_data['hos_region_change'] = [0] *len(combined_data) 

#combined_data['rooms_change'] = [0] *len(combined_data) ##

combined_data['hos_change'] = [0] *len(combined_data) ##

combined_data['grp_first'] = [0] *len(combined_data) 

combined_data['grp_last'] = [0] *len(combined_data) 

combined_data['condition_same'] = [0] *len(combined_data) 

combined_data['condition_worse'] = [0] *len(combined_data) 

combined_data['condition_better'] = [0] *len(combined_data) 

for idx in tnrange(len(combined_data)):
    
    if idx!=0:
        
        if combined_data.loc[idx,'patientid'] == combined_data.loc[idx-1,'patientid']:
            
            if combined_data.loc[idx,'City_Code_Hospital'] != combined_data.loc[idx-1,'City_Code_Hospital']:
                
                combined_data.loc[idx,'city_change'] = 1
            
            if combined_data.loc[idx,'Department'] != combined_data.loc[idx-1,'Department']:
                
                combined_data.loc[idx,'dep_change'] = 1
            
            if combined_data.loc[idx,'Hospital_type_code'] != combined_data.loc[idx-1,'Hospital_type_code']:
                
                combined_data.loc[idx,'hos_type_change'] = 1
                
            if combined_data.loc[idx,'Hospital_region_code'] != combined_data.loc[idx-1,'Hospital_region_code']:
                
                combined_data.loc[idx,'hos_region_change'] = 1
                
            #if combined_data.loc[idx,'Available Extra Rooms in Hospital'] != combined_data.loc[idx-1,'Available Extra Rooms in Hospital']:
                
                #combined_data.loc[idx,'rooms_change'] = 1
                
            if combined_data.loc[idx,'Hospital_code'] != combined_data.loc[idx-1,'Hospital_code']:
                
                combined_data.loc[idx,'hos_change'] = 1
            
            if combined_data.loc[idx,'Type of Admission'] == combined_data.loc[idx-1,'Type of Admission']:
                
                combined_data.loc[idx,'condition_same'] = 1
            
            elif combined_data.loc[idx,'Type of Admission'] > combined_data.loc[idx-1,'Type of Admission']:
                
                combined_data.loc[idx,'condition_worse'] = 1
            
            else:
                combined_data.loc[idx,'condition_better'] = 1
        
        else:
            
            combined_data.loc[idx,'city_change'] = 1
            
            combined_data.loc[idx,'dep_change'] = 1
            
            combined_data.loc[idx,'hos_type_change'] = 1
            
            combined_data.loc[idx,'hos_region_change'] = 1
            
            #combined_data.loc[idx,'rooms_change'] = 1
            
            combined_data.loc[idx,'hos_change'] = 1
            
            combined_data.loc[idx,'grp_first'] = 1
            
            combined_data.loc[idx-1,'grp_last'] = 1
            
            combined_data.loc[idx,'condition_worse'] = 1
    
    else:
        
        combined_data.loc[idx,'city_change'] = 1
        
        combined_data.loc[idx,'dep_change'] = 1
        
        combined_data.loc[idx,'hos_type_change'] = 1
        
        combined_data.loc[idx,'hos_region_change'] = 1
        
        #combined_data.loc[idx,'rooms_change'] = 1
        
        combined_data.loc[idx,'hos_change'] = 1
        
        combined_data.loc[idx,'grp_first'] = 1
        
        combined_data.loc[idx,'condition_worse'] = 1

combined_data.loc[combined_data.index[-1],'grp_last'] = 1

In [14]:
def calculate_probs_features(df_train,colName,newColName,label):

    df = df_train.copy()
    
    probs_stay_df = pd.DataFrame(index= list(range(0,len(df[colName].unique()))),
                                 columns= [colName,newColName + '_p_0',newColName + '_p_1',
                                                   newColName + '_p_2',newColName + '_p_3',
                                                   newColName + '_p_4',
                                                   newColName + '_p_5',newColName + '_p_6',
                                                   newColName + '_p_7',
                                                   newColName + '_p_8',newColName + '_p_9'],
                                          dtype = 'float')

    for i,j in enumerate(train_df[colName].unique()):

        probs_stay_df.iloc[i,0] = j   

        probs_stay_df.iloc[i,1] = len(df.loc[(df[colName]==j) &  (df[label] ==0),:])/len(df.loc[(df[colName]==j) ,:])

        probs_stay_df.iloc[i,2] = len(df.loc[(df[colName]==j) &  (df[label] ==1),:])/len(df.loc[(df[colName]==j) ,:])

        probs_stay_df.iloc[i,3] = len(df.loc[(df[colName]==j) &  (df[label] ==2),:])/len(df.loc[(df[colName]==j) ,:])

        probs_stay_df.iloc[i,4] = len(df.loc[(df[colName]==j) &  (df[label] ==3),:])/len(df.loc[(df[colName]==j) ,:])

        probs_stay_df.iloc[i,5] = len(df.loc[(df[colName]==j) &  (df[label] ==4),:])/len(df.loc[(df[colName]==j) ,:])

        probs_stay_df.iloc[i,6] = len(df.loc[(df[colName]==j) &  (df[label] ==5),:])/len(df.loc[(df[colName]==j) ,:])

        probs_stay_df.iloc[i,7] = len(df.loc[(df[colName]==j) &  (df[label] ==6),:])/len(df.loc[(df[colName]==j) ,:])

        probs_stay_df.iloc[i,8] = len(df.loc[(df[colName]==j) &  (df[label] ==7),:])/len(df.loc[(df[colName]==j) ,:])

        probs_stay_df.iloc[i,9] = len(df.loc[(df[colName]==j) &  (df[label] ==8),:])/len(df.loc[(df[colName]==j) ,:])

        probs_stay_df.iloc[i,10] = len(df.loc[(df[colName]==j) &  (df[label] ==9),:])/len(df.loc[(df[colName]==j) ,:])
        
    return probs_stay_df

In [15]:
train_df = combined_data[combined_data['train']==1]

test_df = combined_data[combined_data['train']==0]

In [16]:
#create prob features
probs_visitor_df = calculate_probs_features(train_df,'Visitors with Patient','Visitors',label)

probs_hospital_code_df = calculate_probs_features(train_df,'Hospital_code','Hospital_code',label) #new

probs_city_code_hospital_df = calculate_probs_features(train_df,'City_Code_Hospital','City_Code_Hospital',label) #new

probs_wards_facilty_df = calculate_probs_features(train_df,'Ward_Facility_Code','Ward_Facility_Code',label) #new

probs_city_code_patient_df = calculate_probs_features(train_df,'City_Code_Patient','City_Code_Patient',label) #new

In [17]:
#merge prob features with train df
train_df = pd.merge(train_df,probs_visitor_df ,on='Visitors with Patient',how='left')

train_df = pd.merge(train_df,probs_hospital_code_df ,on='Hospital_code',how='left')

train_df = pd.merge(train_df,probs_city_code_hospital_df ,on='City_Code_Hospital',how='left')

train_df = pd.merge(train_df,probs_wards_facilty_df ,on='Ward_Facility_Code',how='left')

train_df = pd.merge(train_df,probs_city_code_patient_df ,on='City_Code_Patient',how='left')

In [18]:
#merge prob features with test df
test_df = pd.merge(test_df,probs_visitor_df ,on='Visitors with Patient',how='left')

test_df = pd.merge(test_df,probs_hospital_code_df ,on='Hospital_code',how='left')

test_df = pd.merge(test_df,probs_city_code_hospital_df ,on='City_Code_Hospital',how='left')

test_df = pd.merge(test_df,probs_wards_facilty_df ,on='Ward_Facility_Code',how='left')

test_df = pd.merge(test_df,probs_city_code_patient_df ,on='City_Code_Patient',how='left')

In [19]:
combined_data = pd.concat([train_df,test_df],axis =0)

In [20]:
combined_data.to_csv('combined.csv',index = False)